In [1]:
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import division

In [2]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [3]:
df_sales = pd.read_csv('date_sales.csv')

In [4]:
df_sales.sort_values(by=['date'],inplace =True)

In [5]:
df_diff =df_sales.copy()

In [6]:
df_diff['prev_sales'] = df_diff['sales'].shift(1)

In [7]:
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])
df_diff

,date,sales,prev_sales,diff
540,2015-01-02,468.900,NaN,NaN
7948,2015-01-03,129.552,468.900,-339.348
7946,2015-01-03,362.250,129.552,232.698
7945,2015-01-03,18.840,362.250,-343.410
8307,2015-01-03,137.352,18.840,118.512
...,...,...,...,...
645,2018-12-30,209.300,3.024,206.276
906,2018-12-30,323.136,209.300,113.836
1296,2018-12-30,13.904,323.136,-309.232
1297,2018-12-30,20.720,13.904,6.816


In [8]:
df_supervised = df_diff.drop(['prev_sales'],axis=1)

In [9]:
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

In [10]:
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [11]:
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.25088670775642974


In [12]:
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.4643104328364236


In [13]:
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date'],axis=1)

In [14]:
train_set, test_set = df_model[0:-7].values, df_model[-7:].values

In [15]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [16]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [17]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [18]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
9769/9769 [==============================] - 7s 677us/step - loss: 0.0014
Epoch 2/100
9769/9769 [==============================] - 6s 583us/step - loss: 9.1624e-04
Epoch 3/100
9769/9769 [==============================] - 6s 601us/step - loss: 9.0440e-04
Epoch 4/100
9769/9769 [==============================] - 6s 589us/step - loss: 8.8930e-04
Epoch 5/100
9769/9769 [==============================] - 6s 635us/step - loss: 8.7775e-04
Epoch 6/100
9769/9769 [==============================] - 7s 710us/step - loss: 8.6973e-04
Epoch 7/100
9769/9769 [==============================] - 6s 634us/step - loss: 8.6393e-04
Epoch 8/100
9769/9769 [==============================] - 6s 633us/step - loss: 8.5959e-04
Epoch 9/100
9769/9769 [==============================] - 6s 627us/step - loss: 8.5635e-04
Epoch 10/100
9769/9769 [==============================] - 6s 630us/step - loss: 8.5421e-04
Epoch 11/100
9769/9769 [==============================] - 6s 631us/step - loss: 8.5273e-04
Epoch 12/100

In [19]:
model.save("model.h5",save_format='h5')